# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read the CSV file
cities_df = pd.read_csv("../WeatherPY/output_data/cities.csv")

#Drop unnecessary column
cities_df.drop(cities_df.columns[[0]], axis=1, inplace=True)

#Display the DataFrame
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mayo,38.8876,-76.5119,30.99,86,90,10.36,US,1613670007
1,nome,64.5011,-165.4064,-13.00,70,1,4.81,US,1613669817
2,mataura,-46.1927,168.8643,51.01,92,70,4.00,NZ,1613670009
3,cherskiy,68.7500,161.3000,-29.40,97,0,1.07,RU,1613670011
4,castro,-24.7911,-50.0119,78.28,42,0,5.97,BR,1613670012


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Store 'Lat' and 'Lng' into locations
locations = cities_df[["Lat", "Lng"]]

#Store 'Humidity' into variable
humidity = cities_df['Humidity']

#Create a map
fig = gmaps.figure(zoom_level = 2, center = (30, -30))

#Create a heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity , dissipating = False, max_intensity = 100, point_radius =4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the DataFrame to 10 or less cities
cleaned_cities_df = cities_df.loc[ (cities_df["Cloudiness"] < 10)\
                                  & (cities_df["Max Temp"] < 83)\
                                  & (cities_df["Max Temp"] > 76), :]

#Reset the index
cleaned_cities_df.reset_index(drop=True, inplace=True)

#Display the DataFrame
cleaned_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,castro,-24.7911,-50.0119,78.28,42,0,5.97,BR,1613670012
1,cidreira,-30.1811,-50.2056,78.28,71,1,8.48,BR,1613669918
2,mar del plata,-38.0023,-57.5575,77.00,73,0,11.50,AR,1613670128
3,hambantota,6.1241,81.1185,76.53,83,2,4.03,LK,1613670162
4,aden,12.7794,45.0367,76.48,74,2,9.19,YE,1613669976
5,palmeira,-25.4294,-50.0064,78.40,42,0,5.12,BR,1613670522
6,necochea,-38.5473,-58.7368,80.01,45,0,8.01,AR,1613670597


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Store DataFrame into new variable
hotel_df = cleaned_cities_df

#Create a new column
hotel_df["Hotel Name"] = ''


/Users/ainashtoleu/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [6]:
#Base URL to retrieve data
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#Set the parameters
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
    }

#Use itterows to iterate through DataFrame
for index, row in hotel_df.iterrows():
    
    #Get 'Lat' and 'Lng' from DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    #Add location to parameters
    params["location"] = f"{Lat},{Lng}",
    
    #Assemble URL and make API request
    print(f'Retrieving results for Index:{index}:{Lat},{Lng}.')
    response = requests.get(base_url, params=params).json()
    
    #Extract results
    results = response['results']
    
    try:
        print(f"First Hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print("Missing Hotel ... skipping.")
        
    print("--------------")    


Retrieving results for Index:0:-24.7911,-50.0119.
First Hotel is Central Palace Hotel - Castro (PR).
--------------
Retrieving results for Index:1:-30.1811,-50.2056.


/Users/ainashtoleu/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


First Hotel is Hotel Castelo.
--------------
Retrieving results for Index:2:-38.0023,-57.5575.
First Hotel is Hotel Las Rocas.
--------------
Retrieving results for Index:3:6.1241,81.1185.
First Hotel is The Peacock Beach Hotel Hambantota.
--------------
Retrieving results for Index:4:12.7794,45.0367.
First Hotel is Sheraton Gold Mohur Hotel.
--------------
Retrieving results for Index:5:-25.4294,-50.0064.
First Hotel is Um Hotel Palmeira.
--------------
Retrieving results for Index:6:-38.5473,-58.7368.
First Hotel is Hostel La casa de al lado.
--------------


In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,castro,-24.7911,-50.0119,78.28,42,0,5.97,BR,1613670012,Central Palace Hotel - Castro (PR)
1,cidreira,-30.1811,-50.2056,78.28,71,1,8.48,BR,1613669918,Hotel Castelo
2,mar del plata,-38.0023,-57.5575,77.00,73,0,11.50,AR,1613670128,Hotel Las Rocas
3,hambantota,6.1241,81.1185,76.53,83,2,4.03,LK,1613670162,The Peacock Beach Hotel Hambantota
4,aden,12.7794,45.0367,76.48,74,2,9.19,YE,1613669976,Sheraton Gold Mohur Hotel
5,palmeira,-25.4294,-50.0064,78.40,42,0,5.12,BR,1613670522,Um Hotel Palmeira
6,necochea,-38.5473,-58.7368,80.01,45,0,8.01,AR,1613670597,Hostel La casa de al lado


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))